In [1]:
import json

with open("/scr/ig_pipeline/metadata/fillable_assignments.json") as f:
    data = json.load(f)

In [2]:
import collections

ctr = collections.Counter(data.values())
print(ctr.most_common())

has_assignment = {k for k, v in data.items() if v in {"combined", "ray", "dip"}}

[('combined', 295), ('ray', 201), ('dip', 76), ('manual', 13), ('fixorn', 2), ('changecollision', 1)]


In [3]:
from bddl.knowledge_base import *

Loading BDDL knowledge base... This may take a few seconds.
[nltk_data] Downloading package wordnet to /home/cgokmen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
objects = Object.all_objects()

In [5]:
# Get all the objects that have a fillable synset
fillable_needing = {o.name.split("-")[-1] for o in objects if any(p.name == "fillable" for p in o.category.synset.properties)}

In [6]:
fillable_missing = fillable_needing - has_assignment

In [7]:
print("Fillable needing: ", len(fillable_needing))
print("Fillable assigned:", len(has_assignment))
print("Fillable missing:", len(fillable_missing))

Fillable needing:  591
Fillable assigned: 572
Fillable missing: 19


In [8]:
# Are there any fillables that did not get processed?
print({o.name for o in objects if any(p.name == "fillable" for p in o.category.synset.properties) and o.name.split("-")[-1] not in data})

{'soda_cup-fsfsas', 'litter_box-rwnakn', 'wine_bottle-inkqch', 'trash_can-jlawet', 'clothes_dryer-jasnzj'}


In [9]:
# Export a copy removing everything with no mesh selection
selected_only = {k: v for k, v in data.items() if v in {"combined", "ray", "dip"}}
with open("/scr/ig_pipeline/metadata/fillable_assignments_selected.json", "w") as f:
    json.dump(selected_only, f, indent=4)

In [10]:
# Group all the to-be-fixed objects by provider
grouped = collections.defaultdict(set)
fix_needing = {k for k, v in data.items() if v in ("fix", "changecollision")}
for obj in Object.all_objects():
    model = obj.name.split("-")[-1]
    if model not in fix_needing:
        continue
    grouped[obj.provider].add(obj.name)

for provider, objs in grouped.items():
    print(f"\n{provider}:")
    for obj in sorted(objs):
        print(f"  - {obj}")


scenes/restaurant_brunch:
  - canteen-ouhqnw


In [11]:
not_fillable = {k for k, v in data.items() if v in {"notfillable"}}
synset_fillable_marked_notfillable = fillable_needing & not_fillable
print("\n".join(sorted([x.name for x in Object.all_objects() if x.name.split("-")[-1] in synset_fillable_marked_notfillable])))

In [12]:
fixorn = {k for k, v in data.items() if v in {"fixorn"}}
synset_fillable_marked_fixorn = fillable_needing & fixorn
print("\n".join(sorted([x.name for x in Object.all_objects() if x.name.split("-")[-1] in synset_fillable_marked_fixorn])))

bowl-fedafr
rice_cooker-qatgeb


In [13]:
# Remove the collision assignments of everything in the fix, fixorn and changecollision piles
import glob

import sys
sys.path.append("/scr/ig_pipeline")
        
from b1k_pipeline.utils import parse_name

if False:
    for file in glob.glob("/scr/ig_pipeline/cad/*/*/artifacts/collision_selection.json"):
        # Open each target's collision selection file
        with open(file, "r") as f:
            collision_selections = json.load(f)

        # Find which models are in the fix, fixorn and changecollision piles
        keys_to_remove = []
        for k, v in collision_selections.items():
            model_id = parse_name(k).group("model_id")
            if model_id in data and data[model_id] in {"fix", "fixorn", "changecollision"}:
                keys_to_remove.append(k)

        # Actually remove the collision assignments
        for k in keys_to_remove:
            del collision_selections[k]

        # Write the new collision selection file
        with open(file, "w") as f:
            json.dump(collision_selections, f, indent=4)

In [14]:
# Remove any non-fillable objects from the fillable assignment list
import shutil, glob
if False:
    for obj in objects:
        cat, mdl = obj.name.split("-")
        if mdl not in fillable_needing:
            if mdl in data:
                del data[mdl]
            volume_dirs = glob.glob(f"/scr/ig_pipeline/artifacts/parallels/fillable_volumes/objects/*/{mdl}")
            for vd in volume_dirs:
                shutil.rmtree(vd)

    with open("/scr/ig_pipeline/metadata/fillable_assignments.json", "w") as f:
        json.dump(data, f, indent=4)

In [13]:
# Remove generated meshes for the "fix", "manual", "fixorn" and "changecollision" objects and try again.
import shutil, glob
if True:
    for obj in objects:
        cat, mdl = obj.name.split("-")
        if mdl not in data or data[mdl] not in {"combined", "ray", "dip"}:
            if mdl in data:
                del data[mdl]
            volume_dirs = glob.glob(f"/scr/ig_pipeline/artifacts/parallels/fillable_volumes/objects/*/{mdl}")
            for vd in volume_dirs:
                shutil.rmtree(vd)

    with open("/scr/ig_pipeline/metadata/fillable_assignments.json", "w") as f:
        json.dump(data, f, indent=4)

In [7]:
print("\n".join(f'"{x}",' for x in sorted(fillable_needing)))

"aakcyj",
"ackxiy",
"adciys",
"adiwil",
"adxzhe",
"aefcem",
"aegxpb",
"aeslmf",
"aewpzn",
"aewthq",
"ahbhsd",
"ahtzhp",
"ajzltc",
"akfjxx",
"akusda",
"amhlqh",
"ankfvi",
"apybok",
"arryyl",
"aspeds",
"atgnsc",
"avotsj",
"aynjhg",
"aysfhf",
"azoiaq",
"barzwx",
"bbduix",
"bbewjo",
"bbpraa",
"bdhvnt",
"bedkqu",
"belcml",
"bexgtn",
"bfaqfe",
"bgxzec",
"bnekjp",
"bnobdx",
"bnored",
"bojwlu",
"bovcqx",
"bpwjxr",
"bqpmsv",
"bsdexp",
"bsgybx",
"bupgpj",
"byzaxy",
"bzisss",
"bzsxgw",
"causya",
"cdmmwy",
"cdteyb",
"cdzyew",
"ceaeqf",
"cfdond",
"chjetk",
"cjmezk",
"cjmtvq",
"cjsbft",
"ckkwmj",
"ckxwea",
"cmdagy",
"coqeme",
"cpozxi",
"cprjvq",
"cqdioi",
"crlhmi",
"csanbr",
"csvdbe",
"cvdbum",
"cvyops",
"cwkvib",
"cydfkt",
"cypjlv",
"dafdgk",
"dalyim",
"dbprwc",
"dcleem",
"deudkt",
"dfjcsi",
"dhfqid",
"dhgtvg",
"dhkkfo",
"dhnxww",
"dhseui",
"dhwlaw",
"djgllo",
"dkxddg",
"dnqekb",
"dnvpag",
"dobgmu",
"dpxnlc",
"dtjmai",
"duugbb",
"dwspgo",
"eahqyq",
"eawgwj",
"ecqxgd",
"edfzlt",
"effbnc",
"efkgcw",
